In [1]:
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForMaskedLM


In [ ]:
model_path = "../bin/cloze-model"
tokenizer_name = "microsoft/deberta-v3-base"

In [6]:
df = pd.read_json("../data/strapi-pages.json")
df

,id,documentId,Slug,PageSummary
0,547,s9369wcenw8c0mcc4ax1kn6o,5-experimental-and-clinical-psychologists,"Experimental psychologists, primarily holding ..."
1,553,w24kbrj48qhyh87z0nc0t07v,12-analyzing-the-data,Descriptive statistics are tools used to organ...
2,456,wtyua9xf9617fb4dbb6xrf06,test-page,None
3,584,q2fgxf1dd4zwqcvks4igke2j,18-6-styles-of-management,"Douglas McGregor's influential book, The Human..."
4,571,xzq8f7ep110fn15ykssdrdad,18-2-how-to-understand-intercultural-communica...,"Edward T. Hall, an influential American anthro..."
5,417,fh7f830vgt7eygmzwj3xl2rm,test-page-no-chunks,None
6,330,hzsp4x38uw4jk7jbqnonzh20,page-10,None
7,332,u9xv74nregw4mwlho33uawhy,page-11,None
8,334,s8n50j1aba6ptg00nxpns9kh,page-12,None
9,363,dkpixbdq6pcwyqhvb9fzyc9o,page-1,None
